In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys
sys.path.insert(1, '../py')
import pixel_art_recipes

par = pixel_art_recipes.PixelArtRecipies()
print(['par.{}'.format(fn) for fn in dir(par) if not fn.startswith('_')])

['par.average_dict', 'par.blocks_list', 'par.collate', 'par.color_distance_from', 'par.concrete_and_unglazed_terracotta_list', 'par.concrete_list', 'par.conjunctify_nouns', 'par.convert_minecraft_coords_to_gimp_coords', 'par.convert_minecraft_coords_to_rowcols', 'par.convert_rowcols_to_minecraft_coords', 'par.display_next_minimap', 'par.distance', 'par.dominant_dict', 'par.get_block_name', 'par.get_circle_html', 'par.get_circular_edge_set', 'par.get_column_markup', 'par.get_dictionaries', 'par.get_file_names_dataframe', 'par.get_filename', 'par.get_hex_str', 'par.get_image_html_by_section', 'par.get_it_markup', 'par.get_it_markup_by_groups', 'par.get_map_center', 'par.get_multiples', 'par.get_next_file_names_dataframe', 'par.get_next_recipe', 'par.get_row_markup', 'par.get_rowspan_markup', 'par.get_section_file_path', 'par.get_stack_collation', 'par.get_stack_summary', 'par.get_text_color', 'par.get_text_html_by_section', 'par.glass_and_concrete_and_terracotta_list', 'par.glass_and_con

['black_concrete.png', 'blue_concrete.png', 'brown_concrete.png', 'cyan_concrete.png', 'gray_concrete.png', 'green_concrete.png', 'light_blue_concrete.png', 'light_gray_concrete.png', 'lime_concrete.png', 'magenta_concrete.png', 'orange_concrete.png', 'pink_concrete.png', 'purple_concrete.png', 'red_concrete.png', 'white_concrete.png', 'yellow_concrete.png', 'black_terracotta.png', 'blue_terracotta.png', 'brown_terracotta.png', 'cyan_terracotta.png', 'gray_terracotta.png', 'green_terracotta.png', 'light_blue_terracotta.png', 'light_gray_terracotta.png', 'lime_terracotta.png', 'magenta_terracotta.png', 'orange_terracotta.png', 'pink_terracotta.png', 'purple_terracotta.png', 'red_terracotta.png', 'terracotta.png', 'white_terracotta.png', 'yellow_terracotta.png', 'black_wool.png', 'blue_wool.png', 'brown_wool.png', 'cyan_wool.png', 'gray_wool.png', 'green_wool.png', 'light_blue_wool.png', 'light_gray_wool.png', 'lime_wool.png', 'magenta_wool.png', 'orange_wool.png', 'pink_wool.png', 'purp

In [7]:

pris_blocks_list = [k for k, v in par.dominant_dict.items() if 'pris' in k.lower()]
rgb_dict = {k: v for k, v in par.dominant_dict.items() if k in par.unpowdered_and_unglazed_list}
f = lambda item: np.linalg.norm(np.array(item[1])-pixel)
for block in pris_blocks_list:
    pixel = par.dominant_dict[block]
    file_name = sorted(rgb_dict.items(), key=f)[0][0]
    print(block, file_name)

dark_prismarine.png gray_concrete.png
prismarine.png light_blue_terracotta.png
prismarine_bricks.png light_gray_wool.png


In [4]:

import numpy as np

terra_blocks_list = [k for k, v in par.dominant_dict.items() if 'terra' in k.lower()]
conc_blocks_list = [k for k, v in par.dominant_dict.items() if 'conc' in k.lower()]
rgb_dict = {k: v for k, v in par.dominant_dict.items() if k in conc_blocks_list}
f = lambda item: np.linalg.norm(np.array(item[1])-pixel)
for block in terra_blocks_list:
    pixel = par.dominant_dict[block]
    file_name = sorted(rgb_dict.items(), key=f)[0][0]
    print(block, file_name)

black_glazed_terracotta.png red_concrete.png
black_terracotta.png black_concrete_powder.png
blue_glazed_terracotta.png gray_concrete.png
blue_terracotta.png gray_concrete_powder.png
brown_glazed_terracotta.png brown_concrete_powder.png
brown_terracotta.png brown_concrete.png
cyan_glazed_terracotta.png cyan_concrete.png
cyan_terracotta.png gray_concrete_powder.png
gray_glazed_terracotta.png gray_concrete.png
gray_terracotta.png gray_concrete.png
green_glazed_terracotta.png green_concrete.png
green_terracotta.png green_concrete.png
light_blue_glazed_terracotta.png light_blue_concrete_powder.png
light_blue_terracotta.png light_gray_concrete.png
light_gray_glazed_terracotta.png white_concrete.png
light_gray_terracotta.png light_gray_concrete.png
lime_glazed_terracotta.png lime_concrete_powder.png
lime_terracotta.png green_concrete_powder.png
magenta_glazed_terracotta.png magenta_concrete_powder.png
magenta_terracotta.png light_gray_concrete.png
orange_glazed_terracotta.png orange_concrete_

In [8]:

from_color = (0, 0, 0)
to_rgb_tuple = (1, 1, 1)
color_distance = np.linalg.norm(np.array(from_color) - np.array(to_rgb_tuple))
print(color_distance)
color_distance = par.color_distance_from('black', to_rgb_tuple)
print(color_distance)

1.7320508075688772
1.7320508075688772



----

In [ ]:

art_file_path = '../saves/png/baby_face_134x150.png'
file_names_df = par.get_file_names_dataframe(art_file_path, rgb_dict=par.dominant_dict)
par.show_art_recipe(art_file_path, rgb_dict=par.dominant_dict, blocks_list=None, block_names_df=file_names_df)

In [ ]:

art_file_path = '../saves/png/japanese_maple_bonzai.png'
blocks_list = ['acacia_log.png', 'acacia_log_top.png', 'acacia_planks.png', 'birch_log.png', 'birch_log_top.png', 'birch_planks.png',
               'dark_oak_log.png', 'dark_oak_log_top.png', 'dark_oak_planks.png', 'jungle_log.png', 'jungle_log_top.png', 'jungle_planks.png',
               'oak_log.png', 'oak_log_top.png', 'oak_planks.png', 'spruce_log.png', 'spruce_log_top.png', 'spruce_planks.png']
blocks_list += [k for k, v in par.dominant_dict.items() if 'lea' in k.lower()]
rgb_dict = {k: v for k, v in par.dominant_dict.items() if k in blocks_list}
file_names_df = par.get_file_names_dataframe(art_file_path, rgb_dict=rgb_dict)
par.show_art_recipe(art_file_path, rgb_dict=rgb_dict, blocks_list=blocks_list, block_names_df=file_names_df)


----

In [ ]:

file_names_df.groupby('file_name').count().sort_values('row_number', ascending=False)

In [ ]:

file_names_df.columns.tolist()

In [ ]:

block_names_df.groupby('block_name').count().sort_values('row_number', ascending=False)

In [ ]:

block_names_df.columns.tolist()


----

In [ ]:

par.horizontal_offset = -504
par.vertical_offset = 112

In [ ]:

par.display_next_minimap(x=-365, z=201, art_file_path=art_file_path, file_names_df=file_names_df)

In [ ]:

par.display_next_minimap(x=-198, z=169, art_file_path=art_file_path, file_names_df=file_names_df)

In [ ]:

par.get_map_center()


I have an index with documents with a keyword column(?) and the visualization only shows a 1 for each week for each keyword instead of the total.

In [ ]:

x = -395
z = 307
par.convert_minecraft_coords_to_rowcols(x, z)

In [ ]:

x = -495
z = 207
par.convert_minecraft_coords_to_rowcols(x, z)

In [ ]:

x = -395
z = 207
par.convert_minecraft_coords_to_rowcols(x, z)

In [ ]:

x = -495
z = 307
par.convert_minecraft_coords_to_rowcols(x, z)

In [ ]:

{k: v for k, v in par.dominant_dict.items() if 'polished' in k}

In [ ]:

par.dominant_dict['polished_diorite.png']

In [ ]:

x = -410
z = 222
par.convert_minecraft_coords_to_gimp_coords(x, z)